In [1]:
!pip install pyspark
!pip install findspark

In [2]:
import findspark
findspark.init()

In [3]:
import re
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from pyspark.sql.window import Window

In [4]:
# Expressoes regulares comuns
REGEX_ALPHA    = r'[a-zA-Z]+'
REGEX_INTEGER  = r'[0-9]+'
REGEX_FLOAT    = r'[0-9]+\.[0-9]+'
REGEX_ALPHANUM = r'[0-9a-zA-Z]+'
REGEX_EMPTY_STR= r'[\t ]+$'
REGEX_SPECIAL  = r'[!@#$%&*\(\)_]+'
REGEX_NNUMBER  = r'^N[1-9][0-9]{2,3}([ABCDEFGHJKLMNPRSTUVXWYZ]{1,2})'
REGEX_NNUMBER_INVALID = r'(N0.*$)|(.*[IO].*)'
REGEX_TIME_FMT = r'^(([0-1]?[0-9])|(2[0-3]))([0-5][0-9])$'

In [5]:
# Funcoes auxiiliares

In [6]:
# Criar o contexto do spark
sc = SparkContext()

# Instancia o criador de sessao do spark
spark = (SparkSession.builder
                     .master("local[7]")
                     .appName("Aceleração PySpark - Capgemini"))

In [7]:
# Cria o schema adequado
schema_airports = StructType([
    StructField("faa",  StringType(),  True),
    StructField("name", StringType(),  True),
    StructField("lat",  FloatType(),   True),
    StructField("lon",  FloatType(),   True),
    StructField("alt",  IntegerType(), True),
    StructField("tz",   IntegerType(), True),
    StructField("dst",  StringType(),  True)
])

schema_planes = StructType([
    StructField("tailnum",      StringType(),  True),
    StructField("year",         IntegerType(), True),
    StructField("type",         StringType(),  True),
    StructField("manufacturer", StringType(),  True),
    StructField("model",        StringType(),  True),
    StructField("engines",      IntegerType(), True),
    StructField("seats",        IntegerType(), True),
    StructField("speed",        IntegerType(), True),
    StructField("engine",       StringType(),  True)
])

schema_flights = StructType([
    StructField("year",      IntegerType(), True),
    StructField("month",     IntegerType(), True),
    StructField("day",       IntegerType(), True),
    StructField("dep_time",  StringType(),  True),
    StructField("dep_delay", IntegerType(), True),
    StructField("arr_time",  StringType(),  True),
    StructField("arr_delay", IntegerType(), True),
    StructField("carrier",   StringType(),  True),
    StructField("tailnum",   StringType(),  True),
    StructField("flight",    StringType(),  True),
    StructField("origin",    StringType(),  True),
    StructField("dest",      StringType(),  True),
    StructField("air_time",  IntegerType(), True),
    StructField("distance",  IntegerType(), True),
    StructField("hour",      IntegerType(), True),
    StructField("minute",    IntegerType(), True),
])

In [8]:
# Carrega os dados com as configurações necessárias
df_airports = (spark.getOrCreate().read
                  .format("csv")
                  .option("header", "true")
                  .schema(schema_airports)
                  .load("../data/airports.csv"))

df_planes = (spark.getOrCreate().read
                  .format("csv")
                  .option("header", "true")
                  .schema(schema_planes)
                  .load("../data/planes.csv"))

df_flights = (spark.getOrCreate().read
                  .format("csv")
                  .option("header", "true")
                  .schema(schema_flights)
                  .load("../data/flights.csv"))

## Pergunta 1



In [9]:
df_airports.show()
df_airports.printSchema()

+---+--------------------+---------+-----------+----+---+---+
|faa|                name|      lat|        lon| alt| tz|dst|
+---+--------------------+---------+-----------+----+---+---+
|04G|   Lansdowne Airport|41.130474|  -80.61958|1044| -5|  A|
|06A|Moton Field Munic...| 32.46057|  -85.68003| 264| -5|  A|
|06C| Schaumburg Regional| 41.98934|  -88.10124| 801| -6|  A|
|06N|     Randall Airport| 41.43191|  -74.39156| 523| -5|  A|
|09J|Jekyll Island Air...|31.074472|  -81.42778|  11| -4|  A|
|0A9|Elizabethton Muni...|36.371223| -82.173416|1593| -4|  A|
|0G6|Williams County A...|41.467304| -84.506775| 730| -5|  A|
|0G7|Finger Lakes Regi...|42.883564| -76.781235| 492| -5|  A|
|0P2|Shoestring Aviati...|39.794823| -76.647194|1000| -5|  U|
|0S9|Jefferson County ...| 48.05381|-122.810646| 108| -8|  A|
|0W3|Harford County Ai...|39.566837|   -76.2024| 409| -5|  A|
|10C|  Galt Field Airport| 42.40289| -88.375114| 875| -6|  U|
|17G|Port Bucyrus-Craw...|40.781555|  -82.97481|1003| -5|  A|
|19A|Jac

In [10]:
df_airports = df_airports.withColumn('alt',
                                     F.when(F.col('alt') < 0, 0)
                                     .otherwise(df_airports.alt))



df_airports.groupBy("alt").count().distinct().show()
df_airports.filter(F.col('alt') < 0).count()

+----+-----+
| alt|count|
+----+-----+
| 833|    2|
|1580|    1|
| 148|    2|
| 463|    1|
| 243|    1|
| 540|    1|
| 858|    1|
|1395|    1|
|3179|    1|
| 516|    1|
|  31|    4|
|2776|    1|
|1270|    1|
|2996|    1|
|1339|    1|
| 808|    4|
|  85|    2|
| 137|    3|
| 580|    2|
| 879|    1|
+----+-----+
only showing top 20 rows



0

## Pergunta 2



In [11]:
df_airports = df_airports.withColumn('dst',
                                       F.when(F.col('tz').between(-7,-5),"A")
                                        .otherwise(df_airports.dst))

df_airports.show()


df_airports.groupBy("dst").count().distinct().show()

+---+--------------------+---------+-----------+----+---+---+
|faa|                name|      lat|        lon| alt| tz|dst|
+---+--------------------+---------+-----------+----+---+---+
|04G|   Lansdowne Airport|41.130474|  -80.61958|1044| -5|  A|
|06A|Moton Field Munic...| 32.46057|  -85.68003| 264| -5|  A|
|06C| Schaumburg Regional| 41.98934|  -88.10124| 801| -6|  A|
|06N|     Randall Airport| 41.43191|  -74.39156| 523| -5|  A|
|09J|Jekyll Island Air...|31.074472|  -81.42778|  11| -4|  A|
|0A9|Elizabethton Muni...|36.371223| -82.173416|1593| -4|  A|
|0G6|Williams County A...|41.467304| -84.506775| 730| -5|  A|
|0G7|Finger Lakes Regi...|42.883564| -76.781235| 492| -5|  A|
|0P2|Shoestring Aviati...|39.794823| -76.647194|1000| -5|  A|
|0S9|Jefferson County ...| 48.05381|-122.810646| 108| -8|  A|
|0W3|Harford County Ai...|39.566837|   -76.2024| 409| -5|  A|
|10C|  Galt Field Airport| 42.40289| -88.375114| 875| -6|  A|
|17G|Port Bucyrus-Craw...|40.781555|  -82.97481|1003| -5|  A|
|19A|Jac

## Pergunta 3



In [12]:
df_airports = df_airports.withColumn('dst',
                                       F.when(F.col('dst') == 'U', 'A')
                                       .otherwise(df_airports.dst))

df_airports.show()


df_airports.groupBy("dst").count().distinct().show()

+---+--------------------+---------+-----------+----+---+---+
|faa|                name|      lat|        lon| alt| tz|dst|
+---+--------------------+---------+-----------+----+---+---+
|04G|   Lansdowne Airport|41.130474|  -80.61958|1044| -5|  A|
|06A|Moton Field Munic...| 32.46057|  -85.68003| 264| -5|  A|
|06C| Schaumburg Regional| 41.98934|  -88.10124| 801| -6|  A|
|06N|     Randall Airport| 41.43191|  -74.39156| 523| -5|  A|
|09J|Jekyll Island Air...|31.074472|  -81.42778|  11| -4|  A|
|0A9|Elizabethton Muni...|36.371223| -82.173416|1593| -4|  A|
|0G6|Williams County A...|41.467304| -84.506775| 730| -5|  A|
|0G7|Finger Lakes Regi...|42.883564| -76.781235| 492| -5|  A|
|0P2|Shoestring Aviati...|39.794823| -76.647194|1000| -5|  A|
|0S9|Jefferson County ...| 48.05381|-122.810646| 108| -8|  A|
|0W3|Harford County Ai...|39.566837|   -76.2024| 409| -5|  A|
|10C|  Galt Field Airport| 42.40289| -88.375114| 875| -6|  A|
|17G|Port Bucyrus-Craw...|40.781555|  -82.97481|1003| -5|  A|
|19A|Jac

## Pergunta 4



In [13]:
df_airports = df_airports.withColumn('region',
                                       F.when(F.col('lon') < -124,                       'ALASKA')
                                       .when(F.col('lon').between(-124, -95),            'MAINLAND-WEST')
                                       .when(F.col('lon').between(-95, -54),             'MAINLAND-EAST')
                                       .when((F.col('lon') > -50) | (F.col('lat') > 24), 'OFFSHORE')
                                       .otherwise(None))

df_airports.show()

df_airports.groupBy("region").count().distinct().show()

+---+--------------------+---------+-----------+----+---+---+-------------+
|faa|                name|      lat|        lon| alt| tz|dst|       region|
+---+--------------------+---------+-----------+----+---+---+-------------+
|04G|   Lansdowne Airport|41.130474|  -80.61958|1044| -5|  A|MAINLAND-EAST|
|06A|Moton Field Munic...| 32.46057|  -85.68003| 264| -5|  A|MAINLAND-EAST|
|06C| Schaumburg Regional| 41.98934|  -88.10124| 801| -6|  A|MAINLAND-EAST|
|06N|     Randall Airport| 41.43191|  -74.39156| 523| -5|  A|MAINLAND-EAST|
|09J|Jekyll Island Air...|31.074472|  -81.42778|  11| -4|  A|MAINLAND-EAST|
|0A9|Elizabethton Muni...|36.371223| -82.173416|1593| -4|  A|MAINLAND-EAST|
|0G6|Williams County A...|41.467304| -84.506775| 730| -5|  A|MAINLAND-EAST|
|0G7|Finger Lakes Regi...|42.883564| -76.781235| 492| -5|  A|MAINLAND-EAST|
|0P2|Shoestring Aviati...|39.794823| -76.647194|1000| -5|  A|MAINLAND-EAST|
|0S9|Jefferson County ...| 48.05381|-122.810646| 108| -8|  A|MAINLAND-WEST|
|0W3|Harford

## Pergunta 5



In [ ]:
# Cria os regex's de cada categoria
LIST_AP  = ['Airport', 'Tradeport', 'Heliport', 'Airpor', 'Arpt']
REGEX_AP = r'|'.join(map(lambda x : f'.*{x}*', LIST_AP))

LIST_AD  = ['Aerodrome']
REGEX_AD = r'|'.join(map(lambda x : f'.*{x}*', LIST_AD))

LIST_AK  = ['Airpark', 'Aero Park']
REGEX_AK = r'|'.join(map(lambda x : f'.*{x}*', LIST_AK))

LIST_AS  = ['Station', 'Air Station']
REGEX_AS = r'|'.join(map(lambda x : f'.*{x}*', LIST_AS))

LIST_FL  = ['Field', 'Fld']
REGEX_FL = r'|'.join(map(lambda x : f'.*{x}*', LIST_FL))

In [14]:
df_airports = df_airports.withColumn('type',
              F.when(F.col('name').rlike(REGEX_AP), 'AP')
               .when(F.col('name').rlike(REGEX_AD), 'AD')
               .when(F.col('name').rlike(REGEX_AK), 'AK')
               .when(F.col('name').rlike(REGEX_AS), 'AS')
               .when(F.col('name').rlike(REGEX_FL), 'FL')
               .otherwise('NaN'))
                                     
                                     
df_airports.show()

df_airports.groupBy("type").count().distinct().show()

+---+--------------------+---------+-----------+----+---+---+-------------+----+
|faa|                name|      lat|        lon| alt| tz|dst|       region|type|
+---+--------------------+---------+-----------+----+---+---+-------------+----+
|04G|   Lansdowne Airport|41.130474|  -80.61958|1044| -5|  A|MAINLAND-EAST|  AP|
|06A|Moton Field Munic...| 32.46057|  -85.68003| 264| -5|  A|MAINLAND-EAST|  AP|
|06C| Schaumburg Regional| 41.98934|  -88.10124| 801| -6|  A|MAINLAND-EAST| NaN|
|06N|     Randall Airport| 41.43191|  -74.39156| 523| -5|  A|MAINLAND-EAST|  AP|
|09J|Jekyll Island Air...|31.074472|  -81.42778|  11| -4|  A|MAINLAND-EAST|  AP|
|0A9|Elizabethton Muni...|36.371223| -82.173416|1593| -4|  A|MAINLAND-EAST|  AP|
|0G6|Williams County A...|41.467304| -84.506775| 730| -5|  A|MAINLAND-EAST|  AP|
|0G7|Finger Lakes Regi...|42.883564| -76.781235| 492| -5|  A|MAINLAND-EAST|  AP|
|0P2|Shoestring Aviati...|39.794823| -76.647194|1000| -5|  A|MAINLAND-EAST| NaN|
|0S9|Jefferson County ...| 4

## Pergunta 6



In [15]:
LIST_MILITARY = ['Base', 'Aaf', 'Afs', 'Ahp', 'Afb', 'LRRS', 'Lrrs', 'Arb', 'Naf', 'NAS', 'Nas', 'Jrb', 'Ns', 'As', 'Cgas', 'Angb']
REGEX_MILITARY = r'|'.join(map(lambda x : f'^{x} | {x} | {x}$', LIST_MILITARY))

df_airports = df_airports.withColumn('military',
                                    F.when(F.col('name').rlike(REGEX_MILITARY), True)
                                    .otherwise(False))

df_airports.show()
df_airports.groupBy("military").count().distinct().show()
df_airports.filter(F.col('military')== True).show(200, False)

+---+--------------------+---------+-----------+----+---+---+-------------+----+--------+
|faa|                name|      lat|        lon| alt| tz|dst|       region|type|military|
+---+--------------------+---------+-----------+----+---+---+-------------+----+--------+
|04G|   Lansdowne Airport|41.130474|  -80.61958|1044| -5|  A|MAINLAND-EAST|  AP|   false|
|06A|Moton Field Munic...| 32.46057|  -85.68003| 264| -5|  A|MAINLAND-EAST|  AP|   false|
|06C| Schaumburg Regional| 41.98934|  -88.10124| 801| -6|  A|MAINLAND-EAST| NaN|   false|
|06N|     Randall Airport| 41.43191|  -74.39156| 523| -5|  A|MAINLAND-EAST|  AP|   false|
|09J|Jekyll Island Air...|31.074472|  -81.42778|  11| -4|  A|MAINLAND-EAST|  AP|   false|
|0A9|Elizabethton Muni...|36.371223| -82.173416|1593| -4|  A|MAINLAND-EAST|  AP|   false|
|0G6|Williams County A...|41.467304| -84.506775| 730| -5|  A|MAINLAND-EAST|  AP|   false|
|0G7|Finger Lakes Regi...|42.883564| -76.781235| 492| -5|  A|MAINLAND-EAST|  AP|   false|
|0P2|Shoes

## Pergunta 7



In [16]:
df_airports = df_airports.withColumn('administration',
                                    F.when( F.col('name').rlike(r'.*(International|Intl|Intercontinental).*'), "I")
                                    .when(F.col('name').rlike(r'.*(National|Natl).*'), "N")
                                    .when(F.col('name').rlike(r'.*(Regional|Reigonal|Rgnl|County|Metro|Metropolitan).*'), "R")
                                    .when(F.col('name').rlike(r'.*(Municipal|Muni|City).*'), "M")
                                    .otherwise("NaN")
                                    )

df_airports.show()

df_airports.groupBy("administration").count().distinct().show()

+---+--------------------+---------+-----------+----+---+---+-------------+----+--------+--------------+
|faa|                name|      lat|        lon| alt| tz|dst|       region|type|military|administration|
+---+--------------------+---------+-----------+----+---+---+-------------+----+--------+--------------+
|04G|   Lansdowne Airport|41.130474|  -80.61958|1044| -5|  A|MAINLAND-EAST|  AP|   false|           NaN|
|06A|Moton Field Munic...| 32.46057|  -85.68003| 264| -5|  A|MAINLAND-EAST|  AP|   false|             M|
|06C| Schaumburg Regional| 41.98934|  -88.10124| 801| -6|  A|MAINLAND-EAST| NaN|   false|             R|
|06N|     Randall Airport| 41.43191|  -74.39156| 523| -5|  A|MAINLAND-EAST|  AP|   false|           NaN|
|09J|Jekyll Island Air...|31.074472|  -81.42778|  11| -4|  A|MAINLAND-EAST|  AP|   false|           NaN|
|0A9|Elizabethton Muni...|36.371223| -82.173416|1593| -4|  A|MAINLAND-EAST|  AP|   false|             M|
|0G6|Williams County A...|41.467304| -84.506775| 730| -

# PLANES DATASET

In [17]:
df_planes.show()
df_planes.printSchema()

+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N110UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null

## Pergunta 1



In [18]:
df_planes.filter(F.length(F.col('tailnum')) == 5).show()

+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
|  N1602|1999|Fixed wing multi ...|          BOEING| 767-332|      2|  330| null|Turbo-fan|
|  N1603|1999|Fixed wing multi ...|          BOEING| 767-332|      2|  330| null|Turbo-fan|
|  N1605|1999|Fixed wing multi ...|          BOEING| 767-332|      2|  330| null|Turbo-fan|
|  N1608|2000|Fixed wing multi ...|          BOEING| 767-332|      2|  330| null|Turbo-fan|
|  N1609|2000|Fixed wing multi ...|          BOEING| 767-332|      2|  330| null|Turbo-fan|
|  N3752|2001|Fixed wing multi ...|          BOEING| 737-832|      2|  189| null|Turbo-jet|
|  N3753|2001|Fixed wing multi ...|          BOEING| 737-832|      2|  189| null|Turbo-jet|
|  N3756|2001|Fixed wing multi ...|          BOEING| 737-832|      2|  189| null

In [19]:
'''df_planes = df_planes.withColumn("tailchar",
                     F.when(~F.col('tailnum').rlike(REGEX_NNUMBER), None)
                     .when((F.length(F.col('tailnum'))== 5) & (~F.col('tailnum').substr(4,5).rlike('^[a-zA-Z]+$'))
                           , F.col('tailnum').substr(5,5))
                     .when((F.length(F.col('tailnum'))== 5) & (F.col('tailnum').substr(4,5).rlike('^[a-zA-Z]+$'))
                           , F.col('tailnum').substr(4,5))
                     .when((F.length(F.col('tailnum'))== 6) & (F.col('tailnum').substr(5,6).rlike('^[a-zA-Z]+$'))
                           , F.col('tailnum').substr(5,6))
                     .otherwise(F.col('tailnum').substr(6,6)))
df_planes.show()

df_planes.groupBy("tailchar").count().distinct().show(100)'''

'df_planes = df_planes.withColumn("tailchar",\n                     F.when(~F.col(\'tailnum\').rlike(REGEX_NNUMBER), None)\n                     .when((F.length(F.col(\'tailnum\'))== 5) & (~F.col(\'tailnum\').substr(4,5).rlike(\'^[a-zA-Z]+$\'))\n                           , F.col(\'tailnum\').substr(5,5))\n                     .when((F.length(F.col(\'tailnum\'))== 5) & (F.col(\'tailnum\').substr(4,5).rlike(\'^[a-zA-Z]+$\'))\n                           , F.col(\'tailnum\').substr(4,5))\n                     .when((F.length(F.col(\'tailnum\'))== 6) & (F.col(\'tailnum\').substr(5,6).rlike(\'^[a-zA-Z]+$\'))\n                           , F.col(\'tailnum\').substr(5,6))\n                     .otherwise(F.col(\'tailnum\').substr(6,6)))\ndf_planes.show()\n\ndf_planes.groupBy("tailchar").count().distinct().show(100)'

In [20]:
df_planes = df_planes.withColumn('tailchar', F.regexp_replace(F.col('tailnum'), '[0-9]|^N', ""))

df_planes.groupBy("tailchar").count().distinct().show(100)

+--------+-----+
|tailchar|count|
+--------+-----+
|       K|    7|
|      DZ|   11|
|      PC|    1|
|      UA|  193|
|      PS|    1|
|      AA|   67|
|      AY|    4|
|      MX|    1|
|       F|   16|
|       Q|    3|
|      RR|    1|
|      JC|    1|
|      AT|    4|
|      SW|  280|
|      NW|   99|
|      NA|    1|
|      EV|    4|
|      DQ|    2|
|      UW|  115|
|      VA|   53|
|       E|   10|
|      VJ|    1|
|      QC|    1|
|       T|    2|
|      ZX|    1|
|      LL|    1|
|       B|   28|
|      HK|    1|
|      DX|    1|
|      DN|  118|
|      TW|   15|
|       Y|    5|
|      SA|   21|
|      DL|   61|
|      CA|   21|
|      CT|    1|
|      GB|    1|
|       L|    3|
|       M|    4|
|      BR|    6|
|      NC|    1|
|      PQ|    4|
|      DE|    2|
|      AW|   87|
|       D|   16|
|      YV|    1|
|       C|   20|
|      US|   79|
|       J|    4|
|      FR|   55|
|       Z|    2|
|       A|   46|
|      DA|   46|
|       N|    3|
|       X|    2|
|      AG|   1

## Pergunta 2



In [21]:
df_planes = df_planes.withColumn('year',
                                F.when(F.col('year') == 0, 1996)
                                .otherwise(df_planes.year))

df_planes.groupBy("year").count().distinct().orderBy("year", ascending=True).show()

+----+-----+
|year|count|
+----+-----+
|null|   60|
|1959|    1|
|1963|    1|
|1968|    1|
|1975|    2|
|1976|    1|
|1980|    1|
|1984|    5|
|1985|   15|
|1986|   13|
|1987|   23|
|1988|   29|
|1989|   21|
|1990|   42|
|1991|   48|
|1992|   78|
|1993|   41|
|1994|   39|
|1995|   54|
|1996|   73|
+----+-----+
only showing top 20 rows



## Pergunta 3



In [22]:
# Criando tabela ordenada por Manufacturer e Model
df_ordered = df_planes.groupBy("manufacturer",'model').min('year').orderBy('manufacturer','model')

# Renomeando colunas
df_ordered = df_ordered.withColumnRenamed('manufacturer', 'manufacturer1').withColumnRenamed('model', 'model1') \
             .withColumnRenamed('min(year)', 'year1')

# Criando tabela ordenada somente por manufacturer
df_ordered2 = df_planes.groupBy('manufacturer').min('year').orderBy('manufacturer')

# Renomeando colunas
df_ordered2 = df_ordered2.withColumnRenamed('manufacturer', 'manufacturer2') \
             .withColumnRenamed('min(year)', 'year2')

# Fazendo left join da tabela df_planes com a tabela df_ordered

df_final = df_planes.join(df_ordered, (df_planes.manufacturer == df_ordered.manufacturer1) &
           (df_planes.model == df_ordered.model1),'left')

# Fazendo left join da tabela df_final com a df_ordered2

df_final = df_final.join(df_ordered2, df_final.manufacturer == df_ordered2.manufacturer2 , 'left')

# Modificando a coluna year para inputar os anos

df_final = df_final.withColumn('year',
                              F.when((F.col('year').isNull()) & (F.col('year1').isNull()) & (F.col('year2').isNull()), None )
                              .when((F.col('year').isNull()) & (F.col('year1').isNull()), F.col('year2'))
                              .when(F.col('year').isNull(), F.col('year1'))
                              .otherwise(F.col('year')) 
                              )
# Dropando as colunas criadas pelo join

df_final = df_final.drop('manufacturer1','manufacturer2','model1','year1','year2')

df_final.show()

df_final.select("year","manufacturer","model").filter(F.col('year') == "NaN").show()

df_planes = df_final

df_planes.groupBy("year").count().distinct().orderBy("year", ascending=True).show()

+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+--------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|tailchar|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+--------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      UW|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      US|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      UW|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      UW|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      US|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      UW|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|

In [23]:
df_final.select("year","manufacturer","model").filter(F.col('year').isNull()).show()

+----+---------------+-------------+
|year|   manufacturer|        model|
+----+---------------+-------------+
|null|LAMBERT RICHARD|    FALCON XP|
|null|  BARKER JACK L|ZODIAC 601HDS|
+----+---------------+-------------+



## Pergunta 4



In [24]:
df_planes = df_planes.withColumn('age',  2022 - F.col('year'))

df_planes.show()

df_planes.filter(F.col('age').isNull()).show()

df_planes.groupBy("age").count().distinct().show(100, False)

+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+--------+---+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|tailchar|age|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+--------+---+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      UW| 24|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      US| 23|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      UW| 23|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      UW| 23|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      US| 23|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      UW| 23|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A32

## Pergunta 5



In [25]:
#### arrumar
df_planes = df_planes.withColumn('type',
                                 F.when(F.col('type') == "Fixed wing multi engine", "MULTI_ENG")
                                .when(F.col('type') == "Fixed wing single engine", "SINGLE_ENG")
                                .when(F.col('type') == "Rotorcraft", "ROTORCRAFT")
                                .otherwise(df_planes.type))

df_planes.show()

df_planes.groupBy("type").count().distinct().show(100, False)

+-------+----+---------+----------------+--------+-------+-----+-----+---------+--------+---+
|tailnum|year|     type|    manufacturer|   model|engines|seats|speed|   engine|tailchar|age|
+-------+----+---------+----------------+--------+-------+-----+-----+---------+--------+---+
| N102UW|1998|MULTI_ENG|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      UW| 24|
| N103US|1999|MULTI_ENG|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      US| 23|
| N104UW|1999|MULTI_ENG|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      UW| 23|
| N105UW|1999|MULTI_ENG|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      UW| 23|
| N107US|1999|MULTI_ENG|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      US| 23|
| N108UW|1999|MULTI_ENG|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      UW| 23|
| N109UW|1999|MULTI_ENG|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      UW| 23|
| N110UW|1999|MULTI_ENG|AIRBUS INDUSTRIE|A320-214|      2|  

## Pergunta 6



In [26]:
df_planes.groupBy("manufacturer").count().distinct().show(100, False)

+-----------------------------+-----+
|manufacturer                 |count|
+-----------------------------+-----+
|BARKER JACK L                |1    |
|AIRBUS INDUSTRIE             |401  |
|ROBINSON HELICOPTER CO       |1    |
|SIKORSKY                     |1    |
|BOEING                       |1460 |
|EMBRAER                      |37   |
|CIRRUS DESIGN CORP           |1    |
|MARZ BARRY                   |1    |
|CESSNA                       |4    |
|LAMBERT RICHARD              |1    |
|BOMBARDIER INC               |214  |
|PIPER                        |2    |
|BELL                         |1    |
|MCDONNELL DOUGLAS AIRCRAFT CO|2    |
|MCDONNELL DOUGLAS            |94   |
|KILDALL GARY                 |1    |
|CANADAIR                     |8    |
|AIRBUS                       |397  |
|GULFSTREAM AEROSPACE         |1    |
+-----------------------------+-----+



Errados: BARKER JACK L,
        AIRBUS INDUSTRIE,
        ROBINSON HELICOPTER CO,
        CIRRUS DESIGN CORP,
        BOMBARDIER INC,
        MCDONNELL DOUGLAS AIRCRAFT CO,
        GULFSTREAM AEROSPACE

In [27]:
df_planes = df_planes.withColumn('manufacturer',
                                F.when(F.col('manufacturer') == "BARKER JACK L", "BARKER JACK")
                                .when(F.col('manufacturer') == "AIRBUS INDUSTRIE", "AIRBUS")
                                .when(F.col('manufacturer') == "ROBINSON HELICOPTER CO", "ROBINSON HELICOPTER")
                                .when(F.col('manufacturer') == "CIRRUS DESIGN CORP", "CIRRUS")
                                .when(F.col('manufacturer') == "BOMBARDIER INC", "BOMBARDIER")
                                .when(F.col('manufacturer') == "MCDONNELL DOUGLAS AIRCRAFT CO", "MCDONNELL DOUGLAS")
                                .when(F.col('manufacturer') == "GULFSTREAM AEROSPACE", "GULFSTREAM")
                                .otherwise(df_planes.manufacturer))

df_planes.groupBy("manufacturer").count().distinct().show(100, False)

+-------------------+-----+
|manufacturer       |count|
+-------------------+-----+
|BARKER JACK        |1    |
|GULFSTREAM         |1    |
|CIRRUS             |1    |
|SIKORSKY           |1    |
|BOEING             |1460 |
|EMBRAER            |37   |
|MARZ BARRY         |1    |
|CESSNA             |4    |
|LAMBERT RICHARD    |1    |
|PIPER              |2    |
|BELL               |1    |
|MCDONNELL DOUGLAS  |96   |
|KILDALL GARY       |1    |
|BOMBARDIER         |214  |
|CANADAIR           |8    |
|ROBINSON HELICOPTER|1    |
|AIRBUS             |798  |
+-------------------+-----+



## Pergunta 7



In [28]:
df_planes = df_planes.withColumn('model', F.regexp_replace(F.col('model'), '\s*\([^()]*\)\s*', ""))





df_planes.groupBy("model").count().distinct().orderBy("model", ascending=True).show(1000)
df_planes.show()


+-------------+-----+
|        model|count|
+-------------+-----+
|          150|    1|
|         172M|    1|
|         206B|    1|
|        210-5|    1|
|         421C|    1|
|      737-301|    2|
|      737-3A4|    1|
|      737-3G7|    2|
|      737-3H4|  104|
|      737-3K2|    2|
|      737-3L9|    2|
|      737-3Q8|    4|
|      737-3T5|    1|
|      737-3TO|    2|
|      737-3Y0|    1|
|      737-490|   16|
|      737-4Q8|    9|
|      737-4S3|    1|
|      737-5H4|   10|
|      737-705|    2|
|      737-724|   31|
|      737-732|    2|
|      737-73V|    1|
|      737-76N|   15|
|      737-76Q|    3|
|      737-790|   17|
|      737-7AD|    1|
|      737-7BD|   32|
|      737-7BX|    3|
|      737-7H4|  360|
|      737-7K9|    2|
|      737-7Q8|    1|
|      737-824|  122|
|      737-832|   73|
|      737-890|   60|
|      737-8FH|    1|
|      737-8H4|   81|
|      737-924|   12|
|    737-924ER|  102|
|    737-932ER|   30|
|      737-990|   12|
|    737-990ER|   23|
|      747

## Pergunta 8



In [29]:
df_planes = df_planes.withColumn('speed',
                                F.when((F.col('speed').isNull()) & (F.col('seats').isNotNull())
                                       & (F.col('seats').between(2,500)), (F.ceil(F.col('seats')/0.36)))
                                .otherwise(df_planes.speed))

df_planes = df_planes.withColumn('speed',
                                F.when(~F.col('speed').between(50,150), 0)
                                .otherwise(df_planes.speed))

df_planes.groupBy("speed").count().distinct().show(100, False)
df_planes.show()



+-----+-----+
|speed|count|
+-----+-----+
|0    |2584 |
|112  |1    |
|126  |1    |
|107  |1    |
|56   |1    |
|89   |37   |
|108  |1    |
|90   |2    |
+-----+-----+

+-------+----+---------+------------+--------+-------+-----+-----+---------+--------+---+
|tailnum|year|     type|manufacturer|   model|engines|seats|speed|   engine|tailchar|age|
+-------+----+---------+------------+--------+-------+-----+-----+---------+--------+---+
| N102UW|1998|MULTI_ENG|      AIRBUS|A320-214|      2|  182|    0|Turbo-fan|      UW| 24|
| N103US|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182|    0|Turbo-fan|      US| 23|
| N104UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182|    0|Turbo-fan|      UW| 23|
| N105UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182|    0|Turbo-fan|      UW| 23|
| N107US|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182|    0|Turbo-fan|      US| 23|
| N108UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182|    0|Turbo-fan|      UW| 23|
| N109UW|1999|MULTI_E

## Pergunta 9




In [30]:
df_planes.groupBy("engine").count().distinct().show(100, False)


+-------------+-----+
|engine       |count|
+-------------+-----+
|Turbo-jet    |450  |
|4 Cycle      |1    |
|Turbo-fan    |2127 |
|Turbo-prop   |37   |
|Reciprocating|10   |
|Turbo-shaft  |3    |
+-------------+-----+



In [31]:
df_planes = df_planes.withColumn('engine_type',
                                F.when(F.col('engine') == "Turbo-fan", "FAN")
                                 .when(F.col('engine') == "Turbo-jet", "JET")
                                 .when(F.col('engine') == "Turbo-prop", "PROP")
                                 .when(F.col('engine') == "Turbo-shaft", "SHAFT")
                                 .when(F.col('engine') == "4 Cycle", "CYCLE")
                                 .otherwise(None))
df_planes.show()
df_planes.groupBy("engine_type").count().distinct().show(100, False)


+-------+----+---------+------------+--------+-------+-----+-----+---------+--------+---+-----------+
|tailnum|year|     type|manufacturer|   model|engines|seats|speed|   engine|tailchar|age|engine_type|
+-------+----+---------+------------+--------+-------+-----+-----+---------+--------+---+-----------+
| N102UW|1998|MULTI_ENG|      AIRBUS|A320-214|      2|  182|    0|Turbo-fan|      UW| 24|        FAN|
| N103US|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182|    0|Turbo-fan|      US| 23|        FAN|
| N104UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182|    0|Turbo-fan|      UW| 23|        FAN|
| N105UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182|    0|Turbo-fan|      UW| 23|        FAN|
| N107US|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182|    0|Turbo-fan|      US| 23|        FAN|
| N108UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182|    0|Turbo-fan|      UW| 23|        FAN|
| N109UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182|    0|Turbo-fan|      

# FLIGHTS DATASET

In [32]:
df_flights.show()
df_flights.printSchema()

df_flights.filter(F.col('dep_time')== 'NA').show(48)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
|2014|    1| 15|    1037|        7|    1

## Pergunta 1


In [33]:
df_flights = df_flights.withColumn('hour',
                                  F.when(F.col('hour').isNull(), 0)
                                  .otherwise(df_flights.hour))

df_flights = df_flights.withColumn('minute',
                                  F.when(F.col('minute').isNull(), 0)
                                  .otherwise(df_flights.minute))


df_flights.groupBy("hour").count().distinct().orderBy("hour", ascending=True).show(25)



+----+-----+
|hour|count|
+----+-----+
|   0|  137|
|   1|   17|
|   2|    4|
|   5|  431|
|   6|  899|
|   7|  709|
|   8|  659|
|   9|  456|
|  10|  803|
|  11|  723|
|  12|  539|
|  13|  653|
|  14|  540|
|  15|  468|
|  16|  388|
|  17|  394|
|  18|  570|
|  19|  409|
|  20|  354|
|  21|  281|
|  22|  314|
|  23|  251|
|  24|    1|
+----+-----+



## Pergunta 2



In [34]:
df_flights = df_flights.withColumn('hour',
                                  F.when(F.col('hour') == 24, 0)
                                  .otherwise(df_flights.hour))

df_flights.groupBy("hour").count().distinct().orderBy("hour", ascending=True).show(25)

+----+-----+
|hour|count|
+----+-----+
|   0|  138|
|   1|   17|
|   2|    4|
|   5|  431|
|   6|  899|
|   7|  709|
|   8|  659|
|   9|  456|
|  10|  803|
|  11|  723|
|  12|  539|
|  13|  653|
|  14|  540|
|  15|  468|
|  16|  388|
|  17|  394|
|  18|  570|
|  19|  409|
|  20|  354|
|  21|  281|
|  22|  314|
|  23|  251|
+----+-----+



## Pergunta 3



In [ ]:
df_flights = df_flights.withColumn("dep_datetime", F.expr("make_timestamp(year, month, day, hour, minute, 00)"))

In [35]:
# Arrumar colunas month, day, hour, minute

df_flights = df_flights.withColumn('month',
                      F.when(F.length(F.col('month')) < 2, F.concat(F.lit('0'), F.col('month')))
                      .otherwise(df_flights.month))


df_flights = df_flights.withColumn('day',
                      F.when(F.length(F.col('day')) < 2, F.concat(F.lit('0'), F.col('day')))
                      .otherwise(df_flights.day))


df_flights = df_flights.withColumn('hour',
                      F.when(F.length(F.col('hour')) < 2, F.concat(F.lit('0'), F.col('hour')))
                      .otherwise(df_flights.hour))

df_flights = df_flights.withColumn('minute',
                      F.when(F.length(F.col('minute')) < 2, F.concat(F.lit('0'), F.col('minute')))
                      .otherwise(df_flights.minute))





df_flights.filter(F.length(F.col('month')) < 2).show(100)
df_flights.filter(F.length(F.col('day')) < 2).show(100)
df_flights.filter(F.length(F.col('hour')) < 2).show(100)
df_flights.filter(F.length(F.col('minute')) < 2).show(100)



+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+

+----+-----+---+--------+---------+---

In [36]:
df_flights = df_flights.withColumn('dep_datetime', F.format_string("%s-%s-%s %s:%s:00",
                      F.col('year'),F.col('month'), F.col('day'),F.col('hour'),F.col('minute')).cast("timestamp"))



df_flights.select('dep_datetime').show()
df_flights.printSchema()

+-------------------+
|       dep_datetime|
+-------------------+
|2014-12-08 06:58:00|
|2014-01-22 10:40:00|
|2014-03-09 14:43:00|
|2014-04-09 17:05:00|
|2014-03-09 07:54:00|
|2014-01-15 10:37:00|
|2014-07-02 08:47:00|
|2014-05-12 16:55:00|
|2014-04-19 12:36:00|
|2014-11-19 18:12:00|
|2014-11-08 16:53:00|
|2014-08-03 11:20:00|
|2014-10-30 08:11:00|
|2014-11-12 23:46:00|
|2014-10-31 13:14:00|
|2014-01-29 20:09:00|
|2014-12-17 20:15:00|
|2014-08-11 10:17:00|
|2014-01-13 21:56:00|
|2014-06-05 17:33:00|
+-------------------+
only showing top 20 rows

root
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)
 |-- dep_time: string (nullable = true)
 |-- dep_delay: integer (nullable = true)
 |-- arr_time: string (nullable = true)
 |-- arr_delay: integer (nullable = true)
 |-- carrier: string (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- flight: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: str

## Pergunta 4



In [37]:
df_flights = df_flights.withColumn('dep_time',
                     F.when((F.col('dep_time').isNull()) | (F.col('dep_time') == "NA"),
                            F.format_string("%s%s", F.col('hour'), F.col('minute')))
                     .otherwise(df_flights.dep_time))

df_flights.show()
df_flights.printSchema()

df_flights.groupBy("dep_time").count().distinct().show(25)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|       dep_datetime|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-------------------+
|2014|   12| 08|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|  06|    58|2014-12-08 06:58:00|
|2014|   01| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|2014-01-22 10:40:00|
|2014|   03| 09|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|2014-03-09 14:43:00|
|2014|   04| 09|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|    05|2014-04-09 17:05:00|
|2014|   03| 09|    

## Pergunta 5



In [38]:
df_flights = df_flights.withColumn('dep_delay',
                                  F.when(F.col('dep_delay').isNull(), 0)
                                  .otherwise(df_flights.dep_delay))

df_flights.show()

df_flights.groupBy('dep_delay').count().distinct().show(1000)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|       dep_datetime|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-------------------+
|2014|   12| 08|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|  06|    58|2014-12-08 06:58:00|
|2014|   01| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|2014-01-22 10:40:00|
|2014|   03| 09|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|2014-03-09 14:43:00|
|2014|   04| 09|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|    05|2014-04-09 17:05:00|
|2014|   03| 09|    

## Pergunta 6





In [39]:
df_flights = df_flights.withColumn('arr_delay',
                                  F.when(F.col('arr_delay').isNull() | (F.col('arr_delay') == "NA"), 0)
                                  .otherwise(df_flights.arr_delay))

df_flights.show()

df_flights.filter(F.col("arr_delay")== 0).count()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|       dep_datetime|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-------------------+
|2014|   12| 08|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|  06|    58|2014-12-08 06:58:00|
|2014|   01| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|2014-01-22 10:40:00|
|2014|   03| 09|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|2014-03-09 14:43:00|
|2014|   04| 09|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|    05|2014-04-09 17:05:00|
|2014|   03| 09|    

327

## Pergunta 7



In [40]:
df_flights = df_flights.drop('year','month','day','hour','minute')

df_flights.show()

+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+-------------------+
|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|       dep_datetime|
+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+-------------------+
|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|2014-12-08 06:58:00|
|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|2014-01-22 10:40:00|
|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|2014-03-09 14:43:00|
|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|2014-04-09 17:05:00|
|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|2014-03-09 07:54:00|
|    1037|        7|    1352|        2|     WN| N646SW|    48|   PDX| DEN|     121|     991|2014

## Pergunta 8



In [41]:
df_flights = df_flights.withColumn('air_time_projected',
                                  ((F.col('distance')* 0.1 + 20).cast(IntegerType())))

df_flights.show()

+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+-------------------+------------------+
|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|       dep_datetime|air_time_projected|
+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+-------------------+------------------+
|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|2014-12-08 06:58:00|               115|
|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|2014-01-22 10:40:00|               287|
|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|2014-03-09 14:43:00|                87|
|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|2014-04-09 17:05:00|                76|
|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|  

## Pergunta 9 



In [42]:
df_x = df_flights.groupBy('dest','origin').avg('air_time').orderBy('dest','origin')

df_x = df_x.withColumnRenamed('dest', 'dest1').withColumnRenamed('origin', 'origin1')

df_final2 = df_flights.join(df_x, (df_flights.dest == df_x.dest1) & (df_flights.origin == df_x.origin1),"left")
             
df_flights = df_final2.withColumn('air_time_expected', F.col('avg(air_time)').cast(IntegerType()))

df_flights = df_flights.drop('dest1','origin1','avg(air_time)')

df_flights.toPandas()




,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,dep_datetime,air_time_projected,air_time_expected
0,658,-7,935,-5,VX,N846VA,1780,SEA,LAX,132.0,954,2014-12-08 06:58:00,115,126
1,1040,5,1505,5,AS,N559AS,851,SEA,HNL,360.0,2677,2014-01-22 10:40:00,287,343
2,1443,-2,1652,2,VX,N847VA,755,SEA,SFO,111.0,679,2014-03-09 14:43:00,87,101
3,1705,45,1839,34,WN,N360SW,344,PDX,SJC,83.0,569,2014-04-09 17:05:00,76,85
4,754,-1,1015,1,AS,N612AS,522,SEA,BUR,127.0,937,2014-03-09 07:54:00,113,122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1806,-4,2104,-6,OO,N225AG,3458,SEA,SLC,89.0,689,2014-06-23 18:06:00,88,88
9996,2336,11,452,-13,AA,N3LEAA,1230,SEA,DFW,178.0,1660,2014-08-31 23:36:00,186,195
9997,904,-1,1042,-5,AS,N523AS,360,SEA,SMF,81.0,605,2014-08-08 09:04:00,80,82
9998,1441,26,1820,10,WN,N8647A,2857,SEA,ABQ,133.0,1180,2014-08-29 14:41:00,138,142


## Pergunta 10



In [43]:
df_flights = df_flights.withColumn('air_time',
                                   F.when(F.col('air_time').isNull(), F.greatest(F.col('air_time_projected'), F.col('air_time_expected')))
                                    .otherwise(F.col('air_time')))

df_flights.filter(F.col('air_time') == "NA").count()
df_flights.groupBy('air_time').count().distinct().show(1000)

+--------+-----+
|air_time|count|
+--------+-----+
|     148|   17|
|     243|   14|
|     392|    1|
|      31|   34|
|      85|   96|
|     137|   67|
|     251|   16|
|      65|   11|
|     255|   11|
|      53|    3|
|     133|   96|
|     296|    7|
|      78|   47|
|     322|   10|
|     362|    1|
|     321|    3|
|     108|   59|
|     155|    9|
|     211|   32|
|     193|   57|
|      34|   40|
|     368|    3|
|     101|   94|
|     126|  120|
|     115|   85|
|      81|   74|
|     183|   22|
|      28|   24|
|     210|   41|
|     300|    8|
|      76|   32|
|      27|   23|
|     332|   16|
|      26|   20|
|     271|   18|
|     159|   12|
|     192|   36|
|      44|    4|
|     253|   13|
|     103|   74|
|     329|    8|
|     236|   14|
|     336|    6|
|     350|    3|
|     223|   27|
|      91|   59|
|     409|    1|
|     333|   10|
|     285|   18|
|     222|   20|
|     128|  116|
|     209|   46|
|     330|    7|
|     122|  108|
|     319|   12|
|     230|   1

## Pergunta 11




In [44]:
# Transforma a coluna dep_datetime em segundos com a função unix_timestamp e soma com a coluna air_time
# transformada em segundos, transformando novamente em timestamp depois
df_flights = df_flights.withColumn('sum',
(F.unix_timestamp("dep_datetime") +
(F.col('air_time') * 60).cast('int')).cast('timestamp'))



# Verifica aonde a coluna arr_time está nulo e troca o valor
df_flights = df_flights.withColumn('arr_time',
F.when(F.col('arr_time') == 'NA', F.format_string('%d%d', F.hour(F.col('sum')),
F.minute(F.col('sum'))))
.otherwise(F.col('arr_time')))



# Remove a coluna sum
df_flights = df_flights.drop('sum')



df_flights.toPandas()

,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,dep_datetime,air_time_projected,air_time_expected
0,658,-7,935,-5,VX,N846VA,1780,SEA,LAX,132,954,2014-12-08 06:58:00,115,126
1,1040,5,1505,5,AS,N559AS,851,SEA,HNL,360,2677,2014-01-22 10:40:00,287,343
2,1443,-2,1652,2,VX,N847VA,755,SEA,SFO,111,679,2014-03-09 14:43:00,87,101
3,1705,45,1839,34,WN,N360SW,344,PDX,SJC,83,569,2014-04-09 17:05:00,76,85
4,754,-1,1015,1,AS,N612AS,522,SEA,BUR,127,937,2014-03-09 07:54:00,113,122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1806,-4,2104,-6,OO,N225AG,3458,SEA,SLC,89,689,2014-06-23 18:06:00,88,88
9996,2336,11,452,-13,AA,N3LEAA,1230,SEA,DFW,178,1660,2014-08-31 23:36:00,186,195
9997,904,-1,1042,-5,AS,N523AS,360,SEA,SMF,81,605,2014-08-08 09:04:00,80,82
9998,1441,26,1820,10,WN,N8647A,2857,SEA,ABQ,133,1180,2014-08-29 14:41:00,138,142


## Pergunta 12



In [45]:
df_flights = df_flights.withColumn("haul_duration",
                     F.when(F.col('air_time').between(20,180), "SHORT-HAUL")
                     .when(F.col('air_time').between(180,360), "MEDIUM-HAUL")
                     .when(F.col('air_time') > 360, "LONG-HAUL")
                     .otherwise(None))

df_flights.show()

df_flights.groupBy('haul_duration').count().distinct().show(1000)

+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+-------------------+------------------+-----------------+-------------+
|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|       dep_datetime|air_time_projected|air_time_expected|haul_duration|
+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+-------------------+------------------+-----------------+-------------+
|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|2014-12-08 06:58:00|               115|              126|   SHORT-HAUL|
|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|2014-01-22 10:40:00|               287|              343|  MEDIUM-HAUL|
|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|2014-03-09 14:43:00|                87|              101|   SHORT-HAUL|
|    1705|      

## Pergunta 13




In [46]:
df_flights = df_flights.withColumn('dep_season',
                     F.when(F.col('dep_datetime').between('2013-12-21 21:48:00','2014-03-20 15:33:00'), "WINTER")
                      .when(F.col('dep_datetime').between('2014-03-20 15:33:00','2014-06-21 10:14:00'), "SPRING")
                      .when(F.col('dep_datetime').between('2014-06-21 10:14:00','2014-09-23 02:04:00'), "SUMMER")
                      .when(F.col('dep_datetime').between('2014-09-23 02:04:00','2014-12-21 21:48:00'), "FALL")
                      .when(F.col('dep_datetime').between('2014-12-21 21:48:00','2015-03-20 15:33:00'), 'WINTER')
                      .otherwise(None))

df_flights.show()

df_flights.groupBy('dep_season').count().distinct().show(1000)

df_flights.filter(F.col('dep_season').isNull()).show()

+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+-------------------+------------------+-----------------+-------------+----------+
|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|       dep_datetime|air_time_projected|air_time_expected|haul_duration|dep_season|
+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+-------------------+------------------+-----------------+-------------+----------+
|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|2014-12-08 06:58:00|               115|              126|   SHORT-HAUL|      FALL|
|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|2014-01-22 10:40:00|               287|              343|  MEDIUM-HAUL|    WINTER|
|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|2014-03-09 14:43:00|             

## Pergunta 14



In [47]:
df_flights = df_flights.withColumn('dep_delay_category',
                     F.when(F.col('dep_delay') < 0, "ANTECIPATED")
                     .when(F.col('dep_delay') == 0, "INTIME")
                     .when(F.col('dep_delay').between(1,59),"MINOR")
                     .when(F.col('dep_delay') >= 60, "MAJOR")
                     .otherwise(None))

In [48]:
df_flights.show()

+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+-------------------+------------------+-----------------+-------------+----------+------------------+
|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|       dep_datetime|air_time_projected|air_time_expected|haul_duration|dep_season|dep_delay_category|
+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+-------------------+------------------+-----------------+-------------+----------+------------------+
|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|2014-12-08 06:58:00|               115|              126|   SHORT-HAUL|      FALL|       ANTECIPATED|
|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|2014-01-22 10:40:00|               287|              343|  MEDIUM-HAUL|    WINTER|             MINOR|
|    1443|       -2|    1652| 

### Criando os parquets

In [49]:
(df_airports.select('faa','name', 'lat', 'lon', 'alt', 'tz', 'dst', 'region', 'type', 'military', 'administration')
            .repartition(1) # coalesce
            .write.format("parquet")
            .mode('overwrite')
            .option("header", "true")
            .save("airports_proc.parquet"))


In [50]:
(df_planes.select('tailnum', 'year','type', 'manufacturer', 'model', 'engines', 'seats', 'speed', 'engine', 'tailchar', 'age', 'engine_type')
            .repartition(1) # coalesce
            .write.format("parquet")
            .mode('overwrite')
            .option("header", "true")
            .save('planes_proc.parquet'))
            
            

In [52]:
(df_flights.select(df_flights.columns)
            .repartition(1) # coalesce
            .write.format("parquet")
            .mode('overwrite')
            .option("header", "true")
            .save('flights_proc.parquet'))
            


